“异常”通常被定义为“不符合规范的东西”，因此它有点罕见。

Python中的异常并不罕见。他们到处都有。

实际上，标准Python库中的每个模块都使用异常，Python本身也会在许多不同的情况下引发它们。你们已经看过一些了。

打开一个Python shell，输入代码：

In [1]:
test = [1,2,3]
test[3]

IndexError: list index out of range

`IndexError`是一类“当程序试着去访问在索引类型边界之外的元素时”Python抛出的异常。`IndexError`后面跟的字符串提供了关于是什么导致异常发生的额外信息。

Python的大部分内置异常处理的是这样的情形，即程序试着去执行一条没有合适语义的语句。(我们将在本章稍后处理异常的异常，即那些不能处理错误的异常。) 那些编写和运行过Python程序的读者已遇到大部分这类异常。常见的这类异常有：`TypeError`、`IndexError`、`NameError`、`ValueError`等。

## 7.1 异常处理
到目前为止，我们将异常视为致命事件。当一个异常被引发时，程序终止(在这种情况下，崩溃可能是一个更合适的词)，然后我们返回到代码中，试图找出出错的地方。当引发一个导致程序终止的异常时，我们说**引发了一个未处理的异常**。

异常不需要导致程序终止。当被引发时，异常可以且应当由程序来处理。有时，一个异常被引发是因为程序中存在错误(比如访问不存在的变量)，但很多时候，一个异常是程序员可以也应该预料到的。比如，程序可能试图打开不存在的文件。如果交互程序要求用户输入，用户可能会输入一些不合适的内容。

如果你知道当一行代码执行时可能会引发一个异常时，你应该处理这个异常。在一个写的好的代码里，未被处理的异常应该是这类异常。

考虑如下代码：
```
successFailureRatio = numSuccesses/numFailures
print('The success/failure ratio is', successFailureRatio)
print('Now here')
```
大多数情况下，这段代码都能正常工作，但是如果`numFailures`碰巧为零，它就会失败。尝试除以`0`将导致Python运行时系统引发一个`ZeroDivisionError`异常，并且永远不会到达`print`语句。


In [2]:
numSuccesses = 2
numFailures = 0

successFailureRatio = numSuccesses/numFailures
print('The success/failure ratio is', successFailureRatio)
print('Now here')

ZeroDivisionError: division by zero

比较好的方式是编写如下形式的代码：
```
try:
    successFailureRatio = numSuccesses/numFailures
    print('The success/failure ratio is', successFailureRatio) 
except ZeroDivisionError:
    print('No failures, so the success/failure ratio is undefined.')
print('Now here')
```

In [3]:
try:
    numSuccesses = 2
    numFailures = 0
    successFailureRatio = numSuccesses/numFailures
    print('The success/failure ratio is', successFailureRatio)
except ZeroDivisionError:
    print('No failures, so the success/failure ratio is undefined.')
print('Now here')

No failures, so the success/failure ratio is undefined.
Now here


在进入`try`块时，解释器尝试计算表达式`numsucceeded /numFailures`的值。如果表达式求值成功，程序将表达式的值赋给变量`successfaileratio`，在`try`块的末尾执行`print`语句，然后继续执行`try-except`之后的`print`语句。然而,如果一个`ZeroDivisionError`异常表达式求值，控制立即跳转到`except`块，执行在`except`块里的`print`语句，然后继续执行`try-except`后的`print`语句块。

随堂练习：使用`try-except`块来实现一个满足如下规范的函数。
```
def sumDigits(s):
    """
    Assumes s is a string
    Returns the sum of the decimal digits in s
    For example, if s is 'a2b3c' it returns 5
    """
```

In [7]:
def sumDigits(s):
    """
    Assumes s is a string
    Returns the sum of the decimal digits in s
    For example, if s is 'a2b3c' it returns 5
    """
    sum = 0
    for c in s:
        try:
            val = int(c)
            sum += val
        except ValueError:
            print('遇到一个非整数的字符：', c)
            
    return sum

print(sumDigits('a2b3c'))

遇到一个非整数的字符： a
遇到一个非整数的字符： b
遇到一个非整数的字符： c
5


让我们看另一个示例。考虑如下代码：

In [8]:
val = int(input('Enter an integer: '))
print('The square of the number you entered is', val**2)

Enter an integer: abc


ValueError: invalid literal for int() with base 10: 'abc'

如果用户键入一个可被转换成整数的字符串，则一切都很好。但是，假设用户键入了`abc`，则执行这一行代码会导致Python运行时系统引发一个`ValueError`异常，永远不会到达`print`语句。

程序员写的代码应该像下面这样：

In [13]:
while True:
    val = input('Enter an integer: ')
    try:
        val = int(val)
        print('The square of the number you entered is', val**2)
        break #to exit the while loop
    except ValueError:
        print(val, 'is not an integer')

Enter an integer: abc
abc is not an integer
Enter an integer: 123
The square of the number you entered is 15129


进入循环后，程序询问用户输入一个整数。一旦用户按下了`Enter`键，那么程序会执行`try-except`块。如果`try`块里的前两条语句都没有导致`ValueError`异常，则执行`break`语句，退出`while`循环。但是，如果`try`块中的代码引发了一个`ValueError`异常，控制立即交给`except`块。因此，如果用户输入了一个不能表示为整数的字符串，那么程序将询问用户再次输入。不管用户输入了什么异常，则它将不会导致一个未被处理的异常。

这种修改的缺点是程序文本从两行增加到8行。如果用户被在多处询问输入一个整数，则这样会有问题。当然，这个问题可通过引入一个如下的函数来解决。

def readInt():
    while True:
        val = input('Enter an integer: ')
        try:
            return(int(val)) #convert str to int before returning
        except ValueError:
            print(val, 'is not an integer')
            
readInt()
readInt()

还有更好的，这个函数可被推广来询问任意类型的输入，比如：

In [12]:
def readVal(valType, requestMsg, errorMsg):
    while True:
        val = input(requestMsg + ' ')
        try:
            return(valType(val)) #convert str to valType before returning
        except ValueError:
            print(val, errorMsg)


readVal(int, 'Enter an integer:', 'is not an integer')

Enter an integer: abc
abc is not an integer
Enter an integer: 123


123

函数`readVal`是多态的，它为许多不同类型的参数而工作。因为类型是第一类对象，所以这样的函数用python很容易写。

我们可以像下面这样询问用户输入一个整数：
```
val = readVal(int, 'Enter an integer:', 'is not an integer')
```

异常似乎很不友好，但是考虑另一种情况。比如，当被请求将字符串'abc'转换成一个int类型对象时，类型转换函数int应该做哪些事？它可以返回一个对应于编码字符串的位的整数，但是这不可能跟程序员的意图有联系。作为替代，它可以返回一个特殊的值None。如果它那样做了，则程序员需要插入检查类型转换是否返回None的判断代码。如果忘记了那样的检查，一个程序员将冒着在程序执行期间得到奇怪错误的风险。

有异常后，程序员依然需要包含处理异常的代码。但是，如果程序员忘记包含这样的代码，且异常被引发，则程序会立即停止。这是好事，因为它警告程序的用户有麻烦事发生了，显式的bug比隐式的bug要好很多。而且，它把哪里出错的清楚标记提供给人来用于调试程序。

有可能一块代码引发不止一类异常，可在保留字except后跟一个由异常组成的元组，比如：
```
except (ValueError, TypeError):
```
如果任一列出的异常被引发了，则就进入该`except`块。

作为代替，你可以写一个except块处理所有类型的异常，这允许程序基于引发的异常选择一种行为，比如：
```
except:
```

如果程序员写了如下代码：

In [15]:
def getRatios(vect1, vect2):
    """Assumes: vect1 and vect2 are equal length lists of numbers
       Returns: a list containing the meaningful values of 
           vect1[i]/vect2[i]"""
    ratios = []
    for index in range(len(vect1)):
        try:
            ratios.append(vect1[index]/vect2[index])
        except ZeroDivisionError:
            ratios.append(float('nan')) #nan = Not a Number
        except:
            raise ValueError('getRatios called with bad arguments')
    return ratios

vect1 = [1,2,3]
vect2 = [4,5,6]

print(getRatios(vect1, vect2))

[0.25, 0.4, 0.5]


则如果在try块内引发了任一类型的异常，则就会进入except块。可看图7.1所示。

## 7.2 异常是一种控制流机制
**不要把异常仅当做错误。它是一种方便的、用来简化程序的控制流机制**。

在许多编程语言里，标准的处理错误的方法是让函数返回一个值表明有错误发生了。每个函数调用不得不检查是否返回了那个值。在Python里，更普遍的做法是：当函数不能产生与函数规范一致的结果时，就引发一个异常。

Python的raise语句强制引发一个指定异常。一个raise语句的形式如下：
```
raise exceptionName(arguments)
```
`exceptionName`通常是某个内置异常，比如`ValueError`。但是程序员可以通过创建**子类**来定义新的异常。虽然不同的类型有不同类型的参数，但是大部分时候，这个参数是单个字符串，用来描述异常被触发的理由。